In [13]:
#필요한 라이브러리 가져옴
import numpy as np
import random

#전체 이동 거리를 계산하는 함수
def calculate_total_distance(tour, distance_matrix):
    total_distance = 0 # 전체 이동거리 0으로 초기화 
    for i in range(len(tour) - 1): #입력받은 경로 리스트의 인덱스를 반복하며 , 마지막 인덱스는 제외된다. 
        total_distance += distance_matrix[tour[i]][tour[i + 1]] 
        # 현재 노드와 다음 노드간의 거리를 거리 행렬에서 가져와 total_distance에 더한다.
    total_distance += distance_matrix[tour[-1]][tour[0]]  #마지막 노드에서 시작 노드로의 거리를 추가한다. 
    return total_distance #구한 총 거리를 계산해 나타낸다.

#초기해 생성 
def generate_initial_solution(num_nodes): 
    return [num_nodes - 1] + random.sample(list(range(num_nodes - 1)), num_nodes - 1)
'''노드의 개수를 입력받아 초기해를 생성한다. 
시작 노드는 목적지로 정해지기 때문에 시작 노드를 제외한 다른 노드들의 무작위 순열로 구성된다.
시작 노드는 반드시 마지막에 추가된다.'''

# 이웃해 생성 함수
def generate_neighboring_solution(current_solution):
    i, j = sorted(random.sample(range(1, len(current_solution)), 2))
    new_solution = current_solution[:i] + list(reversed(current_solution[i:j + 1])) + current_solution[j + 1:]
    return new_solution
''' 현재해를 기준으로 이웃해를 생성하는 함수이다.
무작위로 선택된 두 인덱스 사이의 노드 순서를 뒤집어 새로운 해결책을 생성한다.'''

#Tabu search 알고리즘 구현 
def tabu_search(distance_matrix, num_iterations, tabu_size):
    num_nodes = len(distance_matrix)
    current_solution = generate_initial_solution(num_nodes)
    best_solution = current_solution.copy()
    tabu_list = []
    ''' 거리 행렬과 종료조건인 반복횟수,그리고 tabu size를 입력받는다.
    노드의 개수를 주어진 거리 행렬대로 설정한다. 
    current_solution과 best_solution 위에서 생성한 초기해로 초기화한다.
    tabu list를 빈 리스트로 초기화해준다.
    '''
    for iteration in range(num_iterations): #입력한 반복 횟수만큼 반복 진행
        neighboring_solution = generate_neighboring_solution(current_solution)
        #현재 solution을 기반으로 이웃해들을 생성한다. 

        #tabu list 업데이트
        tabu_list.append(current_solution)

        candidate_solutions = [current_solution, neighboring_solution]
        candidate_solutions_distances = [calculate_total_distance(sol, distance_matrix) for sol in candidate_solutions]
        ''' current_solution과 이웃해(neighboring_solution)을 포함하는 리스트 candidat_solutions을 만든다.
        주어진 이웃해들의 총 거리합을 구해 candidate_solutions_distances리스트에 저장한다.'''

        best_index = np.argmin(candidate_solutions_distances)
        

        current_solution = candidate_solutions[best_index]
        ''' 계산한 이웃해들의 거리 리스트에서 최솟값이 있는 인덱스를 반환해 best_index에 저장한다.
        current_solution(현재 solution)을 선택한 최적의 이웃해로 업데이트 한다. '''


        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)
        ''' tabu list안에 있는 값들이 설정한 tabu size보다 큰 경우, 
        리스트의 0번째(가장 처음에 들어온) 값을 삭제한다. 
        '''
        # 최적해와 현재해 사이의 총 거리값을 비교한다.
        if calculate_total_distance(current_solution, distance_matrix) < calculate_total_distance(best_solution, distance_matrix):
            best_solution = current_solution.copy()
        # 만약 현재해의 총 거리 값이 더 작으면 best solution을 업데이트 해준다.

        # 현재 반복에서의 최적해의 거리 값과 그때의 방문 노선에 대한 정보를 출력한다.
        print(f"Iteration {iteration + 1}: Best Distance = {calculate_total_distance(best_solution, distance_matrix)}")
        print(f"Order of Nodes Visited: {best_solution + [best_solution[0]]}")  #출발지로 되돌아 오도록 

    return best_solution #최적해를 리턴한다.


# 거리 행렬 
distance_matrix = [[0, 6276, 1144, 7317, 8888, 5023, 8497, 3435, 1291],
[6331, 0, 5658, 3904, 6799, 1995, 6408, 3429, 7145],
[1252, 5548, 0, 6589, 8674, 4295, 8283, 2891, 1611],
[7421, 3145, 6585, 0, 6598, 3236, 8677, 5036, 8072],
[9262, 7115, 8589, 5907, 0, 7469, 1287, 6360, 10076],
[5139, 1917, 4303, 2958, 7020, 0, 6629, 2267, 5790],
[8502, 6355, 7829, 6440, 480, 6709, 0, 5600, 9316],
[4222, 3328, 3549, 5227, 6208, 2136, 5817, 0, 5036],
[1428, 6992, 1748, 8105, 9462, 5811, 9071, 4089, 0]]


num_iterations = 10000 
#반복횟수 파라미터
tabu_size = 15
# tabu 사이즈 파라미터


best_tour = tabu_search(distance_matrix, num_iterations, tabu_size)
#거리 행렬과 반복횟수, tabu사이즈를 바탕으로 최종적으로 구한 최적해를 best_tour 변수에 할당한다. 
print("Best Tour:", best_tour + [best_tour[0]])  # Include the return to the starting node
# 가장 좋았던 노선 루트를 출력한다.

print("Best Distance:", calculate_total_distance(best_tour, distance_matrix))
# 가장 좋았던 노선의 총 거리 값을 출력한다.

Iteration 1: Best Distance = 50414
Order of Nodes Visited: [8, 4, 1, 7, 5, 0, 6, 3, 2, 8]
Iteration 2: Best Distance = 50414
Order of Nodes Visited: [8, 4, 1, 7, 5, 0, 6, 3, 2, 8]
Iteration 3: Best Distance = 49957
Order of Nodes Visited: [8, 7, 1, 4, 5, 0, 6, 3, 2, 8]
Iteration 4: Best Distance = 49957
Order of Nodes Visited: [8, 7, 1, 4, 5, 0, 6, 3, 2, 8]
Iteration 5: Best Distance = 47679
Order of Nodes Visited: [8, 7, 4, 1, 5, 0, 6, 3, 2, 8]
Iteration 6: Best Distance = 47679
Order of Nodes Visited: [8, 7, 4, 1, 5, 0, 6, 3, 2, 8]
Iteration 7: Best Distance = 47679
Order of Nodes Visited: [8, 7, 4, 1, 5, 0, 6, 3, 2, 8]
Iteration 8: Best Distance = 47679
Order of Nodes Visited: [8, 7, 4, 1, 5, 0, 6, 3, 2, 8]
Iteration 9: Best Distance = 47592
Order of Nodes Visited: [8, 7, 4, 3, 6, 0, 5, 1, 2, 8]
Iteration 10: Best Distance = 47592
Order of Nodes Visited: [8, 7, 4, 3, 6, 0, 5, 1, 2, 8]
Iteration 11: Best Distance = 40645
Order of Nodes Visited: [8, 0, 6, 3, 4, 7, 5, 1, 2, 8]
Iteratio